In [1]:
#引入本部分所需要的包

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

app_tr_path="..\\data\\home-credit-default-risk\\application_train.csv"
app_te_path="..\\data\\home-credit-default-risk\\application_test.csv"
bur_path="..\\data\\home-credit-default-risk\\bureau.csv"
bur_bal_path="..\\data\\home-credit-default-risk\\bureau_balance.csv"
pos_path="..\\data\\home-credit-default-risk\\POS_CASH_balance.csv"
cre_path="..\\data\\home-credit-default-risk\\credit_card_balance.csv"
pre_path="..\\data\\home-credit-default-risk\\previous_application.csv"
ins_path="..\\data\\home-credit-default-risk\\installments_payments.csv"

hom_path="..\\data\\home-credit-default-risk\\HomeCredit_columns_description.csv"# 列描述表


### 1. **application_{train|test}.csv**

In [2]:
#导入数据

app_tr=pd.read_csv(app_tr_path)
app_te=pd.read_csv(app_te_path)

In [3]:
#查看数据的基本信息

app_tr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Columns: 122 entries, SK_ID_CURR to AMT_REQ_CREDIT_BUREAU_YEAR
dtypes: float64(65), int64(41), object(16)
memory usage: 286.2+ MB


In [4]:
#缺失值处理

#查看数据各属性的缺失值个数
np.sum(app_tr.isnull(),axis = 0)

SK_ID_CURR                        0
TARGET                            0
NAME_CONTRACT_TYPE                0
CODE_GENDER                       0
FLAG_OWN_CAR                      0
                              ...  
AMT_REQ_CREDIT_BUREAU_DAY     41519
AMT_REQ_CREDIT_BUREAU_WEEK    41519
AMT_REQ_CREDIT_BUREAU_MON     41519
AMT_REQ_CREDIT_BUREAU_QRT     41519
AMT_REQ_CREDIT_BUREAU_YEAR    41519
Length: 122, dtype: int64

In [5]:
#用各属性的最高频率值来填补缺失值

def fillnan_most(data):
    df = data.copy(deep=True)
    for col in df.columns: 
        most = df[col].value_counts().index[0]
        df[col] = df[col].fillna(most)
    return df

app_tr_1=fillnan_most(app_tr)
app_te_1=fillnan_most(app_te)

In [6]:
#查看缺失值处理后各属性的缺失值个数

np.sum(app_te_1.isnull(),axis = 0)

SK_ID_CURR                    0
NAME_CONTRACT_TYPE            0
CODE_GENDER                   0
FLAG_OWN_CAR                  0
FLAG_OWN_REALTY               0
                             ..
AMT_REQ_CREDIT_BUREAU_DAY     0
AMT_REQ_CREDIT_BUREAU_WEEK    0
AMT_REQ_CREDIT_BUREAU_MON     0
AMT_REQ_CREDIT_BUREAU_QRT     0
AMT_REQ_CREDIT_BUREAU_YEAR    0
Length: 121, dtype: int64

In [7]:
#异常值处理

#对于数值属性，用盒图来判断异常值，将异常值设为空值，并用各属性的最高频率值来填补

def box_outlier(data):
    df = data.copy(deep=True)
    for col in df.select_dtypes(exclude='object').columns:             # 对数值属性的每一列分别用盒图进行判断
        Q1 = df[col].quantile(q=0.25)       # 下四分位
        Q3 = df[col].quantile(q=0.75)       # 上四分位
        low_whisker = Q1 - 1.5 * (Q3 - Q1)  # 下边缘
        up_whisker = Q3 + 1.5 * (Q3 - Q1)   # 上边缘
        
        # 寻找异常点,获得异常点索引值，将异常值设为空值
        rule = (df[col] > up_whisker) | (df[col] < low_whisker)  
        out = df[col].index[rule]
        df.loc[out,col]=np.nan
    return df

app_tr_2=box_outlier(app_tr_1)
app_te_2=box_outlier(app_te_1)

In [8]:
#用各属性的最高频率值来填补空值

app_tr_3=fillnan_most(app_tr_2)
app_te_3=fillnan_most(app_te_2)

In [9]:
#将数据导出

app_tr_3.to_csv("../data/app_tr.csv")
app_te_3.to_csv("../data/app_te.csv")
